# Python for Petroleum Engineering Calculations

## Newton Raphson's Method

In [ ]:
def ritchie(fn, x, tol = 0.0001, max_iteration = 100):
    for i in range(max_iteration):
        x_new = x - fn[0](x) / fn[1](x)
        if abs(x_new - x) < tol: break
        x = x_new
    return x_new, i

y = [lambda x: 2*x**3 -9.5*x +7.5, lambda x: 6*x**2 -9.5]
x,n = ritchie(y,5)
print(x,n)

## Molecular Weight of components

In [ ]:
def calculate_molecular_weight():
    component = {"c1" : 0.83,"c2" : 0.06,"c3" : 0.03,"n-c4" : 0.03,"n-c5" : 0.02,"c6" : 0.01,"c7+" : 0.03}
    mw = {"c1" : 16.0,"c2" : 30.1,"c3" : 44.1,"n-c4" : 58.1,"n-c5" : 72.2,"c6" : 84.0,"c7+" : 161}
    Total_molecular_weight = sum(component[a] * mw[a] for a in component)
    return Total_molecular_weight

Total_molecular_weight = calculate_molecular_weight()
print(Total_molecular_weight)

## Sutton's Methodology

In [ ]:
from math import sqrt, exp
component = {"c1" : 0.83,"c2" : 0.06,"c3" : 0.03,"n-c4" : 0.02,"n-c5" : 0.02,"c6" : 0.01,"c7+" : 0.03}
mw = {"c1" : 16.04,"c2" : 30.07,"c3" : 44.09,"n-c4" : 58.12,"n-c5" : 72.15,"c6" : 86.17,"c7+" : 161}
Tci = {"c1" : 343.33,"c2" : 549.93,"c3" : 666.06,"n-c4" : 765.62,"n-c5" : 845.60,"c6" : 923.0,"c7+" : 1188.915}
Pci = {"c1" : 666.4,"c2" : 706.5,"c3" : 616.4,"n-c4" : 550.6,"n-c5" : 488.6,"c6" : 483.0,"c7+" : 318.4096}

def calculate_molecular_weight():
    Total_molecular_weight = sum(component[a] * mw[a] for a in component)
    return Total_molecular_weight

Total_molecular_weight = calculate_molecular_weight()
print("Molecular weight", Total_molecular_weight)

def calculate_specific_gravity():
    Yg_each = {a:mw[a] / 28.97 for a in mw}
    return Yg_each
Yg_each = calculate_specific_gravity()

Tpc_sutton ={}
Ppc_sutton = {}



Tpc_sutton = sum(Tci[a] * component[a] for a in Tci)
Ppc_sutton = sum(Pci[a] * component[a] for a in Pci)


Ek = (1189.0 / sqrt(318.4)) * ((0.3129 * 0.03) - (4.8156 * 0.03 **2) + (27.3751 * 0.03 **3) )
Fj = (1/3 * (0.03 * 1189.0 / 318.4)) + 2/3 * ((0.03 * 1189.0 / 318.4) **0.5) **2
Ej = ((0.6081 * Fj) + (1.1325 * Fj **2) - (14.004 * 0.03 * Fj) + (64.434 * Fj * 0.03 **2))
total_sumk = sum((component[a] * Tci[a]) / sqrt(Pci[a]) for a in component)
K = total_sumk
total_sumj = sum(component[a] * (Tci[a] / Pci[a]) for a in component)
J = 1/3 * (total_sumj) + 2/3 * ((total_sumj) **0.5) **2
K_prime = K - Ek
J_prime = J - Ej

corrected_Tpc = (K_prime)**2 / J_prime
corrected_Ppc = corrected_Tpc / J_prime

pressure = 2000
temperature = 610
Ppr_corrected = pressure / corrected_Ppc 
Tpr_corrected = temperature / corrected_Tpc


from numpy import exp
def Hall_yarborough_corrected():
    Ppr_corrected = 3.17200895304848
    Tpr_corrected = 1.4691247911037875
    t = 1/Tpr_corrected
    
    def calculate_gas(f, g, Y_corrected, X1, X2, X3, X4, tol=1e-12, max_iterations=100):
        for i in range(max_iterations):
            Y_new = Y_corrected - f/g
            if abs(Y_new - Y_corrected) < tol: break
        return Y_new

   
    X1 = -0.06125 * Ppr_corrected*t* exp(-1.2 * (1-t)**2)
    X2 = (14.76 * t - 9.76*t**2 + 4.58*t**3)
    X3 = (90.7*t - 242.2*t**2 + 42.4*t**3)
    X4 = (2.18 + 2.82*t)
    
    Y_corrected = 0.06125*Ppr_corrected*t**(-1.2 * (1-t)**2)
    
    f = X1 +((Y_corrected + Y_corrected**2 + Y_corrected**3 + Y_corrected**4)/(1-Y_corrected)**3) - (X2) * Y_corrected**2 + (X3) * Y_corrected**X4 
    g = ((1 + 4*Y_corrected + 4*Y_corrected**2 - 4*Y_corrected**3 + Y_corrected**4)/(1-Y_corrected)**4) - 2*(X2)*Y_corrected + (X3)*(X4)*Y_corrected**(X4 -1)
    
    Y_corrected = calculate_gas(f, g, Y_corrected, X1, X2, X3, X4, tol=1e-12, max_iterations=100)

    z_corrected = ((0.06125*Ppr_corrected*t)/Y_corrected) * exp(-1.2 * (1-t)**2)
    return z_corrected

Hall_yarborough_corrected()


Yg_total = Total_molecular_weight / 28.96
Tpc = 168 + 325 * (Yg_total) - 12.5 * (Yg_total)**2
Ppc = 677 + 15 * (Yg_total)  - 37.5 * (Yg_total)**2
P = 2000
T = 610

Ppr = P/Ppc
Tpr = T/Tpc


from numpy import exp
def Hall_yarborough():
    Ppr = 3.018640274088295
    Tpr = 1.3994682651009254
    t = 1/Tpr
    
    def calculate_gas(f, g, Y, X1, X2, X3, X4, tol=1e-12, max_iterations=50):
        for i in range(max_iterations):
            Y_new = Y - f/g
            if abs(Y_new - Y) < tol: break
        return Y_new

   
    X1 = -0.06125 * Ppr * t* exp(-1.2 * (1-t)**2)
    X2 = (14.76 * t - 9.76*t**2 + 4.58*t**3)
    X3 = (90.7*t - 242.2*t**2 + 42.4*t**3)
    X4 = (2.18 + 2.82*t)
    
    Y = 0.06125*Ppr*t**(-1.2 * (1-t)**2)
    
    f = X1 +((Y + Y**2 + Y**3 + Y**4)/(1-Y)**3) - (X2) * Y**2 + (X3) * Y**X4 
    g = ((1 + 4*Y + 4*Y**2 - 4*Y**3 + Y**4)/(1-Y)**4) - 2*(X2)*Y + (X3)*(X4)*Y**(X4 -1)
    
    Y_new = calculate_gas(f, g, Y, X1, X2, X3, X4, tol=1e-12, max_iterations=100)

    z = ((0.06125*Ppr*t)/Y_new) * exp(-1.2 * (1-t)**2)
    return z

Hall_yarborough()

def density_corrected():
    R = 10.73
    ρ_corrected = (P * calculate_molecular_weight()) / (Hall_yarborough_corrected() * R * T)
    return ρ_corrected
ρ_corrected = density_corrected()

def density():
    R = 10.73
    ρ = (P * calculate_molecular_weight()) / (Hall_yarborough() * R * T)
    return ρ
ρ = density()

absolute_difference = abs(density_corrected() - density())
percentage_error = (absolute_difference / density_corrected()) * 100

print(""" """)
print("adjusted Ppr", Ppr_corrected)
print("adjusted Tpr", Tpr_corrected)
print("adjusted z_factor", Hall_yarborough_corrected())
print("Adjusted density",ρ_corrected)
print(""" """)
print("unadjusted Ppr",Ppr)
print("unadjusted Tpr",Tpr)
print("unadjusted z_factor",Hall_yarborough())
print("Unadjusted density",ρ)
print(""" """)
print("Error", percentage_error)


## Standings Correlation for Oil viscosity Calculations

In [ ]:
def calculate_viscosity_dead_oil(API, t):
    A = 10**(0.43 + (8.33/API))
    Uod = (0.32 + (1.8**7 /API**4.53)) * (360 / t + 200)**A
    return Uod
    

def calculate_viscosity_gas_saturated_crude_oil(Uod, Rs):
    a = Rs * (2.2*10**-7*Rs - 7.4*10**-4)
    c = 8.62*10**-5*Rs
    d = 1.10*10**-3*Rs
    e = 3.74*10**-3*Rs
    b = 0.68/10**c + 0.25/10**d + 0.062/10**e
    Uob = 10**a*Uod**b
    return Uob
    
def calculate_viscosity_undersaturated_crude_oil(Uob, p, pb):
    Uo = Uob + 0.001*(p - pb)*(0.024 * Uob**1.6 + 0.38 * Uob**0.56)
    return Uo
    
print("1. Viscosity of Dead oil")
print("2. Viscosity of gas saturated crude oil")
print("3. Viscosity of undersaturated crude oil")

choice = input("input your choice:")

if choice == "1":
    print("DEAD OIL")
    API = float(input("input the API value given:"))
    t = float(input("input your temperature value:"))
    Uod = calculate_viscosity_dead_oil(API, t)
    print(Uod)
    
elif choice == "2":
    print("GAS SATURATED OIL")
    Rs = float(input("input solution gas oil ratio given:"))
    API = float(input("input the API value given:"))
    t = float(input("input your temperature value:"))
    Uod = calculate_viscosity_dead_oil(API, t)
    Uob = calculate_viscosity_gas_saturated_crude_oil(Uod, Rs)
    print(Uob)
    
elif choice == "3":
    print("UNDERSATURATED OIL")
    API = float(input("input the API value given:"))
    t = float(input("input your temperature value:"))
    Rs = float(input("input solution gas oil ratio given:"))
    Uod = calculate_viscosity_dead_oil(API, t)
    Uob = calculate_viscosity_gas_saturated_crude_oil(Uod, Rs)
    p = float(input("input the pressure given:"))
    pb = float(input("input the bubble point pressure given:"))
    Uo = calculate_viscosity_undersaturated_crude_oil(Uob, p, pb)
    print(Uo)
    
else:
    print("Wrong input!")